In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [10]:
import glob
import tensorflow as tf
import cv2
import numpy as np
from tqdm.auto import tqdm
from skimage import transform
from ganime.visualization.videos import display_videos
from joblib import Parallel, delayed

In [5]:
source_folder = "../../../data/kny/scenes/*/*.mp4"

In [6]:
video_paths = sorted(glob.glob(source_folder))

In [7]:
def resize_video(video, new_height=64, new_width=128):
    n_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = np.zeros((n_frames, new_height, new_width, 3))
    i = 0
    while True:
        ret, frame = video.read()
        if ret == True:
            resized = cv2.resize(frame,(new_width, new_height),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
            frames[i] = resized
        else:
            break
        i += 1
    return frames

In [8]:
def frame_count(video_path, manual=False):
    def manual_count(handler):
        frames = 0
        while True:
            status, frame = handler.read()
            if not status:
                break
            frames += 1
        return frames 

    cap = cv2.VideoCapture(video_path)
    # Slow, inefficient but 100% accurate method 
    if manual:
        frames = manual_count(cap)
    # Fast, efficient but inaccurate method
    else:
        try:
            frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        except:
            frames = manual_count(cap)
    cap.release()
    return frames

In [11]:
all_frame_numbers = Parallel(n_jobs=-1)(delayed(frame_count)(path) for path in tqdm(video_paths))
all_frame_numbers = np.array(all_frame_numbers)

  0%|          | 0/17338 [00:00<?, ?it/s]

In [12]:
def get_value_to_split(array):
    n_elements = array.shape[0]
    range_min = 15
    range_max = 25
    
    lowest_remainder = range_max
    lowest_id = range_max
    for i in range(range_min, range_max):
        remainder = n_elements % i
        if remainder <= lowest_remainder:
            lowest_id = i
            lowest_remainder = remainder
    return lowest_id

In [13]:
def cut_array(array, number):
    for i in range(array.shape[0] // number):
        start = i * number
        end = (i+1) * number
        if array.shape[0] - end < number:
            end = array.shape[0]
            yield start, end, len(array[start:end])
        else:
            yield start, end, len(array[start:end])

In [27]:
array = np.arange(33)
for i in cut_array(array, get_value_to_split(array)):
    print(i)

(0, 16, 16)
(16, 33, 17)


In [105]:
get_value_to_split(array)

20

In [29]:
np.median(all_frame_numbers)

54.0

In [7]:
def load_video(path):
    video = cv2.VideoCapture(path)
    video = resize_video(video)
    #video = tf.convert_to_tensor(video)
    return video
    

In [8]:

videos = Parallel(n_jobs=-5)(delayed(load_video)(path) for path in tqdm(video_paths))
videos = tf.ragged.stack(videos)

  0%|          | 0/17338 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Process LokyProcess-24:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py", line 455, in _process_worker
    result_queue.put(pid)
  File "/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/backend/queues.py", line 239, in put
    with self._wlock:
  File "/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/backend/synchronize.py", line 104, in __enter__
    return self._semlock.acquire()
KeyboardInterrupt
Process LokyProcess-99:
Traceback (m